## A Molecule Classifier
In this section will demonstrate how to use TorchDrug to create a molecule classifier. Specifically, TorchDrug provides us with a large collection of popular datasets and models for drug discovery and graph representation learning.
We will use ClinTox dataset in the following section. This dataset requires to predict whether a molecule is toxic in clinic trials, and whether it is approved by FDA

In [1]:
import torch
import torchDrug as td
from torchdrug import datasets
from torch import utils

dataset = datasets.ClinTox('../data/')
lengths = [int(0.8 * len(dataset)), int(1.0 * len(dataset))]
lengths += [len(dataset) - sum(lengths)]
train_set, valid_set, test_set = utils.data.random_split(dataset, lengths)

19:52:26   Downloading http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/clintox.csv.gz to C:\Users\Leanne/data/clintox.csv.gz
19:52:27   Extracting C:\Users\Leanne/data/clintox.csv.gz to C:\Users\Leanne/data\clintox.csv


Constructing molecules from SMILES:   0%|          | 0/1484 [00:00<?, ?it/s]D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `*`
  warnings.warn("Unknown value `%s`" % x)
[19:52:27] Explicit valence for atom # 0 N, 5, is greater than permitted
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Tc`
  warnings.warn("Unknown value `%s`" % x)
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Al`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   5%|▌         | 75/1484 [00:00<00:01, 744.56it/s]D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Ca`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:  10%|█         

## define a GNN for classification
We define a GNN to encode the molecule graphs. We use a Graph Isomorphism network with 4 hidden layers

In [5]:
from torchdrug import core, models, tasks

dataset = datasets.ClinTox('./data/')
model = models.GIN(input_dim=dataset.node_feature_dim, hidden_dims=[256, 256, 256, 256],
                   short_cut=True, batch_norm=True, concat_hidden=True)

20:11:11   Downloading http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/clintox.csv.gz to ./data/clintox.csv.gz
20:11:16   Extracting ./data/clintox.csv.gz to ./data\clintox.csv


Constructing molecules from SMILES:   0%|          | 0/1484 [00:00<?, ?it/s]D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `*`
  warnings.warn("Unknown value `%s`" % x)
[20:11:16] Explicit valence for atom # 0 N, 5, is greater than permitted
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Tc`
  warnings.warn("Unknown value `%s`" % x)
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Al`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   6%|▌         | 86/1484 [00:00<00:01, 854.84it/s]D:\anaconda\envs\d2l\lib\site-packages\torchdrug\data\feature.py:42: UserWarning: Unknown value `Ca`
  warnings.warn("Unknown value `%s`" % x)
D:\anaconda\envs\d2l\lib\site-packages\torchdrug\da

The model is simply a neural network without any training target. To adapt it for classification, we wrap it with a property prediction module. We define the classification task by the binary cross entropy criterion.

In [6]:
task = tasks.PropertyPrediction(model, task=dataset.tasks, criterion="bce", metric=("auprc", "auroc"))

In [7]:
optimizer = torch.optim.Adam(task.parameters(), lr=1e-3)
solver = core.Engine(task, train_set, valid_set, test_set, optimizer, batch_size=1024, gpus=[0])
solver.train(num_epoch=100)

20:11:45   Preprocess training set
20:11:45   {'batch_size': 1024,
 'class': 'core.Engine',
 'gpus': [0],
 'gradient_interval': 1,
 'log_interval': 100,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'eps': 1e-08,
               'foreach': None,
               'lr': 0.001,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'tasks.PropertyPrediction',
          'criterion': 'bce',
          'graph_construction_model': None,
          'metric': ('auprc', 'auroc'),
          'mlp_batch_norm': False,
          'mlp_dropout': 0,
          'model': {'activation': 'relu',
                    'batch_norm': True,
                    'class': 'models.GIN',
                    'concat_hidden': True,
                    'edge_input_dim': None,
                    'eps': 0,
                  

ImportError: DLL load failed while importing torch_ext: 找不到指定的模块。